In [1]:
import numpy as np
from beiras_aux import LoadText,predict_next_chars


In [ ]:
window_size = 100
step_size = 1
X,y,chars,chars_to_indices,indices_to_chars=LoadText('Beiras.txt',window_size,step_size);

In [16]:
from keras.layers import TimeDistributed
from keras.layers import Dense, Activation,GRU
from keras.optimizers import RMSprop
from keras.models import Sequential

def create_gru_distributed_model(chars):
    num_chars = len(chars)
    model= Sequential()
    # 1 Layer .- LSTM layer 1 should be an LSTM module with 200 hidden units
    model.add(GRU(200,input_shape = (None,num_chars),return_sequences=True))
    # 2 Layer .-  Dense, with number chars unit and softmax activation
    model.add(GRU(200,return_sequences=True))
    model.add(TimeDistributed(Dense(num_chars)))
    model.add(Activation('softmax'))
    # initialize optimizer
    optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
    # compile model --> make sure initialized optimizer and callbacks - as defined above - are used
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    return model

In [ ]:
model=create_gru_distributed_model(chars)
model.summary()
model.fit(X_distributedtime, y_distributedtime, batch_size=500, nb_epoch=30,verbose = 1)

# save weights
model.save_weights('model_weights/best_beiras_gru_distributed_textdata_weights.hdf5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_3 (GRU)                  (None, None, 200)         153600    
_________________________________________________________________
gru_4 (GRU)                  (None, None, 200)         240600    
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 55)          11055     
_________________________________________________________________
activation_2 (Activation)    (None, None, 55)          0         
Total params: 405,255
Trainable params: 405,255
Non-trainable params: 0
_________________________________________________________________


/home/aind2/anaconda3/envs/beiras-rnn/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/30
  61500/1074280 [>.............................] - ETA: 1198s - loss: 2.5717

In [ ]:
# function that uses trained model to predict a desired number of future characters
def predict_next_chars_distributed(model,input_chars,num_to_predict):     
    # create output
    predicted_chars = ''
    for i in range(num_to_predict):
        # convert this round's predicted characters to numerical input    
        x_test = np.zeros((1, window_size, len(chars)))
        for t, char in enumerate(input_chars):
            x_test[0, t, chars_to_indices[char]] = 1.

        # make this round's prediction
        test_predict = model.predict(x_test,verbose = 0)[0][window_size-1]
        

        # translate numerical prediction back to characters
        r = np.argmax(test_predict)                           # predict class of each test input
        d = indices_to_chars[r] 

        # update predicted_chars and input
        predicted_chars+=d
        input_chars+=d
        input_chars = input_chars[1:]
    return predicted_chars

def print_predicctions_distributed(model,weights_file):
    start_inds = [100,1000,5000]

    # load in weights
    model.load_weights(weights_file)
    for s in start_inds:
        start_index = s
        input_chars = text_clean[start_index: start_index + window_size]

        # use the prediction function
        predict_input = predict_next_chars_distributed(model,input_chars,num_to_predict = 100)

        # print out input characters
        print('------------------')
        input_line = 'input chars = ' + '\n' +  input_chars + '"' + '\n'
        print(input_line)

        # print out predicted characters
        line = 'predicted chars = ' + '\n' +  predict_input + '"' + '\n'
        print(line)  

In [ ]:
model=create_gru_distributed_model(chars)
print_predicctions_distributed(model,'model_weights/best_beiras_gru_distributed_textdata_weights.hdf5')


Consigue bos resultados con so 100000 carazteres, pero a base de comerse a memoria, na miña maquina aws con 100000 casi se come toda a memoria.